#### **Преобразование txt файлов в csv** 

In [ ]:
import os
import codecs
import csv

# преобразуем наши текстовые файлы в csv файл
PATH = "C:\\Users\\Asus\\Desktop\\nan"  # путь к папке с файлами
file_names = os.listdir(PATH) 

    csv_writer = csv.writer(csv_file, delimiter=',')
    n_docs = 0
    for name in file_names:
        if name.startswith('en'):
            n_docs += 1
            with codecs.open(PATH + "/" + name, encoding = 'utf-8') as f:
                result = f.read()
                num = ''.join([str(s) for s in list(name) if s.isdigit()])
                name_ru = f'ru_paint_{num}.txt'
                f_ru = open(f"C:\\Users\\Asus\\Desktop\\nan\\{name_ru}", encoding='utf-8')
                result_ru = f_ru.read()
                csv_writer.writerow([result]+[result_ru])
print(n_docs) 

In [ ]:
import pandas as pd

# добавляем названия к колонкам
df = pd.read_csv("paint0204.csv", header=None)
df.to_csv("paint0204.csv", header=["en", "ru"], index=False)

### Предобработка текста

In [ ]:
import pandas as pd

# длина документов до обработки
corpus = pd.read_csv("paint0204.csv")
corpus['len_en'] = corpus['en'].apply(lambda x: len(str(x).split()))
corpus['len_ru'] = corpus['ru'].apply(lambda x: len(str(x).split()))
corpus.head()


In [ ]:
import re 
import nltk
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

import spacy
nlp = spacy.load("en_core_web_sm")

STW_PATH = '/Users/ASUS/Desktop/swl_optimum.txt' 
file = open(STW_PATH, encoding = 'utf-8')
stw_ru = file.read().split()[1:]
stw_en = set(stopwords.words('english'))

def english(text):
    """
    Преобработка текстов на английском языке. 
    Приводим к нижнему регистру, 
    удаляем цифры и знаки препинания,
    удаляем стоп-слова, 
    нормализация слов (лемматизация с spacy), 
    выделяем биграммы?
    """
    text = str(text).lower()
    text = re.sub(r'[^\w\s]|\d','', text)
    text = word_tokenize(text)
    
    text = [word for word in text if not word in stw_en]
    text = ' '.join(text)
    hel = nlp(text)
    text = [token.lemma_ for token in hel]

    # text = [ps.stem(w) for w in text]
    
    print("**")
    return ' '.join(text)


def russian(text):
    """
    Преобработка текстов на русском языке. 
    Приводим к нижнему регистру, 
    удаляем цифры и знаки препинания,
    удаляем стоп-слова, 
    нормализация слов (скорее всего лемматизация через pymorphy2), 
    выделяем биграммы?
    """
    res = ''
    text = str(text).lower()
    text = re.sub(r'[^\w\s]|\d','', text)
    text = word_tokenize(text)
    
    for word in text:                         
        p = morph.parse(word)[0]
        res = res + ' ' + p.normal_form # лемматизация
    text = [word for word in res.split() if not word in stw_ru]
    
    print("**")
    return ' '.join(text)

In [ ]:
corpus['en'] = corpus['en'].apply(english)
corpus['ru'] = corpus['ru'].apply(russian)
corpus.head()

In [ ]:
# длина документов после обрабоки
corpus['len_en_stem'] = corpus['en'].apply(lambda x: len(str(x).split()))
corpus['len_ru_lem'] = corpus['ru'].apply(lambda x: len(str(x).split()))
corpus.head()

In [ ]:
# corpus.to_csv("lemstem_2try.csv", header=["en", "ru", "len_en", "len_ru", "len_en_stem", "len_ru_lem"], index=False)

## Выделение биграмм и триграмм


### Для русского языка

In [ ]:
text_ru = []
for index, row in corpus.iterrows():
        text_ru.append(row['ru'].split())

In [ ]:
from gensim.models import Phrases
bigram = Phrases(text_ru, scoring='npmi', threshold=0.4, min_count=5) 

for idx in range(len(text_ru)):
    for token in bigram[text_ru[idx]]:
        if '_' in token:
            # print(token)
            text_ru[idx].append(token)

In [ ]:
texts = [' '.join(text) for text in text_ru]
corpus['ru'] = texts

### Для английского языка


In [ ]:
text_en = []
for index, row in corpus.iterrows():
        text_en.append(row['en'].split())

In [ ]:
bigram = Phrases(text_en, scoring='npmi', threshold=0.4, min_count=5) 

for idx in range(len(text_en)):
    for token in bigram[text_en[idx]]:
        if '_' in token:
            # print(token)
            text_en[idx].append(token)

In [ ]:
texts_en = [' '.join(text) for text in text_en]
corpus['en'] = texts_en

In [ ]:
corpus['len_en_bi'] = corpus['en'].apply(lambda x: len(str(x).split()))
corpus['len_ru_bi'] = corpus['ru'].apply(lambda x: len(str(x).split()))

In [ ]:
corpus.head()

In [ ]:
corpus.to_csv("wiki_prep.csv", header=["en", "ru", "len_en", "len_ru", "len_en_stem", "len_ru_lem", "len_en_bi", "len_ru_bi"], index=False)